In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shishang_fushi=pd.DataFrame(columns=columns)
channel_shishang_fushi

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').click()  #选择“时尚”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'时尚'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择“服饰”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text
sub_channel

'服饰'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [ ]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shishang_fushi.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-23 00:27:46.566571


<ipython-input-9-3bf486c8607c>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-3bf486c8607c>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-3bf486c8607c>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1C7411Y72Z视频属性的抓取
BV号为1C7411Y72Z的视频属性抓取结束


<ipython-input-9-3bf486c8607c>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为197411a7zf视频属性的抓取
BV号为197411a7zf的视频属性抓取结束
开始BV号为1i7411r7zo视频属性的抓取
BV号为1i7411r7zo的视频属性抓取结束
开始BV号为1X7411v7qY视频属性的抓取
BV号为1X7411v7qY的视频属性抓取结束
开始BV号为1J7411674J视频属性的抓取
BV号为1J7411674J的视频属性抓取结束
开始BV号为1q7411B7hG视频属性的抓取
BV号为1q7411B7hG的视频属性抓取结束
开始BV号为1qJ411p7dV视频属性的抓取
BV号为1qJ411p7dV的视频属性抓取结束
开始BV号为1y7411a71x视频属性的抓取
BV号为1y7411a71x的视频属性抓取结束
开始BV号为1f7411r7TY视频属性的抓取
BV号为1f7411r7TY的视频属性抓取结束
开始BV号为1u7411r7oc视频属性的抓取
BV号为1u7411r7oc的视频属性抓取结束
开始BV号为167411q7gk视频属性的抓取
BV号为167411q7gk的视频属性抓取结束
开始BV号为1c7411q75h视频属性的抓取
BV号为1c7411q75h的视频属性抓取结束
开始BV号为1GJ411V7Vi视频属性的抓取
BV号为1GJ411V7Vi的视频属性抓取结束
开始BV号为127411e7Jg视频属性的抓取
开始BV号为127411e7Jg视频属性的抓取
BV号为127411e7Jg的视频属性抓取结束
开始BV号为1X7411k77E视频属性的抓取
BV号为1X7411k77E的视频属性抓取结束
开始BV号为1iJ411H72i视频属性的抓取
BV号为1iJ411H72i的视频属性抓取结束
开始BV号为1RJ411L72E视频属性的抓取
BV号为1RJ411L72E的视频属性抓取结束
开始BV号为1wJ411j7C4视频属性的抓取
BV号为1wJ411j7C4的视频属性抓取结束
开始BV号为1Z7411z7Ks视频属性的抓取
BV号为1Z7411z7Ks的视频属性抓取结束
开始BV号为1i7411i7jY视频属性的抓取
BV号为1i7411i7jY的视频属性抓取结束
开始BV号为1WJ41177y8视频属性的抓取
BV号为1WJ41177y8的视频属性抓取结束
开始BV号为1VJ41157mB

In [29]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-29-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [11]:
channel_shishang_fushi.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [12]:
channel_shishang_fushi.to_json('channel_shishang_fushi.json')

In [20]:
channel_shishang_fushi

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1C7411Y72Z,1月,韩国东东,个人认证,118.7万,287,旗袍长袍让韩国人闭麦？？看民国风的超真实反应！,时尚,服饰,普通视频,自制,"[宋轶, 张曼玉, 倪妮, 陈坤, 张若昀, 刘诗诗, 吴亦凡, 王一博, 服饰, 时尚, ...",07:22,218.0万,4.3万,5813,4305,311,1557,9103,否
197411a7zf,1月,慕_MXY,无,4586,4,被老姐女装,时尚,服饰,普通视频,自制,"[服饰, 美妆, 时尚, 服饰, 可爱的男孩子, 女装, 化妆, 女装大佬]",00:09,143.2万,1.6万,689,4975,917,785,251,否
1i7411r7zo,1月,青木美沙子-official-,个人认证,31.2万,53,我是青木美沙子，我入驻b站啦，给大家拜年〜,时尚,服饰,活动作品,自制,"[LOLITA FASHION, 服饰, 时尚, 服饰, 青木美沙子, bilibili新星...",01:11,122.4万,8.7万,1.8万,1.2万,3745,4918,2993,否
1X7411v7qY,1月,长安五分甜,无,1.5万,29,【长安】155cm43kg//过期JK的不同长度袜子的对比 | 31cm小腿围如何选择显瘦袜...,时尚,服饰,活动作品,自制,"[JK制服, 服饰, 时尚, 服饰, 搭配, 时尚, 测评, 时尚穿搭, bilibili新...",04:15,113.9万,4.4万,1.9万,4.1万,9267,2847,3174,否
1J7411674J,1月,zettaranc,个人认证,177.3万,128,李宁为什么越来越贵？还越卖越火？丨Z说球鞋,时尚,服饰,活动作品,自制,"[球鞋, 潮流, 篮球, 服饰, 体育, 科普, 时尚, 服饰, 全能打卡挑战, 李宁, 运...",39:40,112.1万,10.6万,12.2万,2.4万,1.1万,4814,4.1万,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15z4y1o7Wh,10月,青鸢Cyan-,个人认证,49.4万,105,这该死的心动❤️JK制服鞋大秀| 快给女朋友买一双！,时尚,服饰,活动作品,自制,"[JK制服, 服饰, 时尚, 服饰, 双十一好物博物馆, 服装搭配, 搭配, 日常, 试穿,...",01:46,18.8万,9833,3814,9891,1076,316,166,否
1Np4y1Y71h,10月,樊不烦F,无,1.9万,58,秋冬加绒睡衣合集～无敌软糯！性价比也太高了吧！质量绝了！！！,时尚,服饰,普通视频,自制,"[购物分享, 服饰, 时尚, 服饰, 女装, 学生党, 睡衣, 平价]",09:02,18.8万,5007,1251,7396,591,282,109,否
1vf4y1i7Nb,10月,BettySays,个人认证,57.7万,85,【如何找准个人风格】10分钟教你挑选理想衣橱单品 ！零基础 | 断舍离 | 时尚 | Bet...,时尚,服饰,普通视频,自制,"[种草, 服饰, 时尚, 服饰, 服饰搭配, 设计, 时尚, 女装, 日常, 搭配, 教程]",12:09,18.6万,1.3万,6988,1.0万,599,219,5187,否
1wz4y1o7H5,10月,蒜末凉子,个人认证,20.2万,135,拥抱秋天/焦糖巧克力色系穿搭/10件单品反复搭配胶囊衣橱,时尚,服饰,普通视频,自制,"[服饰, 时尚, 服饰, 搭配]",03:21,18.6万,5690,1751,5180,251,271,188,否


In [10]:
1

1